<a href="https://colab.research.google.com/github/0yacinek0/nlp/blob/practice/%5Bgithub_nlp%5D_6_%ED%8A%B8%EB%9E%9C%EC%8A%A4%ED%8F%AC%EB%A8%B8%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이 튜토리얼에서는 BERT 모델(논문)을 이용해서 연습해본다. 이를 위해 Huggingface의 트랜스포머 라이브러리를 이용하여 훈련된 단어 벡터를 얻는다. 그후 bidirectional GRU 모델을 이용하여 최종 출력을 얻는다.

### Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pwd # 현재 경로 확인 

'/content'

In [ ]:
%cd /content/drive/MyDrive/논문/sentimentP/dataset

/content/drive/MyDrive/논문/sentimentP/dataset


In [ ]:
ls

Mecab-ko-for-Google-Colab/  ratings_train.txt  train_data.csv
ratings_test.txt            test_data.csv


### 필요한 패키지와 랜덤시드 설정

In [ ]:
# !pip install torchtext==0.10.1 
!pip install torch==1.8.0 torchtext==0.9.0
"""
아마 예전 코드를 실행하시면서 지금은 사라진 객체를 생성하시려다가 생기는 오류 같습니다.

torchtext 버전을 0.10.x 이나 그 이전으로 낮춰보시는 것은 어떠실까요?
pip install torchtext==0.10.1 과 같은 식으로 버전을 지정하셔서 설치하실 수 있습니다.

https://stackoverflow.com/questions/73055161/importerror-cannot-import-name-unicode-csv-reader-from-torchtext-utils

"""

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 89.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.


'\n아마 예전 코드를 실행하시면서 지금은 사라진 객체를 생성하시려다가 생기는 오류 같습니다.\n\ntorchtext 버전을 0.10.x 이나 그 이전으로 낮춰보시는 것은 어떠실까요?\npip install torchtext==0.10.1 과 같은 식으로 버전을 지정하셔서 설치하실 수 있습니다.\n\nhttps://stackoverflow.com/questions/73055161/importerror-cannot-import-name-unicode-csv-reader-from-torchtext-utils\n\n'

# 전처리
* 레이블의 갯수가 두개 이상인 데이터의 분류를 해보자. 그런데 한글 데이터셋을 못구해서... 일단 영어 데이터로 진행하겠다. 이 경우는 6개의 클래스가 있는 TREC 데이터를 분석할 것이며, 따라서 LABEL 필드에서 dtype을 선언할 필요가 없다.

In [ ]:
# 코랩 내 다운로드 https://wikidocs.net/94600
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
/content/drive/MyDrive/논문/sentimentP/dataset/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 41.5 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-01-29 18:01:22--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c5:2ef4, 2406:da00:ff00::6b17:d1f5, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downl

### 랜덤시드 고정

In [ ]:
import torch
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### 트랜스포머의 사전 훈련 벡터를 이용하자. BERT 모델 중 다중언어를 지원하는 'bert-base-multilingual-cased' 토크나이저를 이용

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.7 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

### tokenizer.vocab에는 사전 훈련된 단어들이 들어있다. 갯수를 확인

In [ ]:
len(tokenizer.vocab)

119547

### 토큰화 하기 위해서는 tokenizer.tokenize 메서드를 이용

In [ ]:
tokens = tokenizer.tokenize('우리 사이엔 낮은 담이 있어 서로의 진심을 안을 수가 없어요')
print(tokens)

['우', '##리', '사', '##이', '##엔', '낮', '##은', '담', '##이', '있어', '서로', '##의', '진', '##심을', '안', '##을', '수', '##가', '없', '##어', '##요']


### 이런 토큰을 인덱스로 변경하려면 tokenizer.convert_tokens_to_ids 메서드를 이용

In [ ]:
indexes = tokenizer.convert_tokens_to_ids(tokens)
print(indexes)

[9604, 12692, 9405, 10739, 86933, 8992, 10892, 9064, 10739, 45893, 67324, 10459, 9708, 86904, 9521, 10622, 9460, 11287, 9555, 12965, 48549]


### 트랜스포머 모델은 문장의 시작과 끝을 의미하는 특별한 토큰들 또한 훈련시켜놓았다. 물론 패딩과 미지의 토큰에 대해서도 훈련되어 있다.

In [ ]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


### 특별한 토큰들의 인덱스를 알아보기

In [ ]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [ ]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


### 트랜스포머 모델은 문장 최대 길이가 지정되어 있음
* max_model_input_sizes를 통해 확인

In [ ]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-multilingual-cased']
print(max_input_length)

512


### 기존에는 은전한닢 토크나이저를 이용하였으나 여기서는 직접 함수를 만들어서 최대 길이로 자르고 여기에 추가로 문장 시작 토큰과 마지막 토큰을 고려하여 두개를 추가로 뺀다.

In [ ]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    return tokens

### 이제 필드를 정의한다. 트랜스포머는 배치 차원을 맨 앞에 놓아야 하므로 batch_first = True 옵션을 넣어야 한다. 그리고 단어장을 이미 만들었으므로 use_vocab = False를 선언한다. 토크나이저로는 위에서 정의한 tokenize_and_cut 함수를 이용한다. preprocessing 인수는 토크나이징 된 이후 적용할 함수를 입력하는 곳이며, 여기서는 토큰을 인덱스로 변환하는 메서드를 넣는다. 마지막으로 특수 토큰들을 인덱스로 선언한다(이미 후처리에서 인덱스로 변환되었으므로).

In [ ]:
import torchtext
from torchtext import data
from torchtext.legacy import data # 추가

TEXT = torchtext.legacy.data.Field(batch_first = True,
                 use_vocab = False,
                 tokenize = tokenize_and_cut,
                 preprocessing = tokenizer.convert_tokens_to_ids,
                 init_token = init_token_idx,
                 eos_token = eos_token_idx,
                 pad_token = pad_token_idx,
                 unk_token = unk_token_idx)

LABEL = torchtext.legacy.data.LabelField(dtype = torch.float)

### 데이터 불러온 후 분리 

In [ ]:
fields = {'text': ('text',TEXT), 'label': ('label',LABEL)}
# dictionary 형식은 {csv컬럼명 : (데이터 컬럼명, Field이름)}

In [ ]:
train_data, test_data = data.TabularDataset.splits(
                            path = '.',
                            train = 'train_data.csv',
                            test = 'test_data.csv',
                            format = 'csv',
                            fields = fields,  
)
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [ ]:
print(f'훈련 데이터 갯수: {len(train_data)}')
print(f'검증 데이터 갯수: {len(valid_data)}')
print(f'테스트 데이터 갯수: {len(test_data)}')

훈련 데이터 갯수: 104996
검증 데이터 갯수: 44999
테스트 데이터 갯수: 49997


### 실제 데이터가 인덱스로 변환되었는지 확인

In [ ]:
print(vars(train_data.examples[10]))

{'text': [9574, 119439, 10459, 18622, 42815, 12638, 30842, 14423, 48418, 11664, 80331, 10622, 9056, 35866, 11018, 119259, 12178, 117, 100, 119, 9428, 12030, 85903, 16439, 9729, 26737, 17342], 'label': '0'}


### 실제 문장 확인

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[10])['text'])
print(tokens)

['영', '##혼', '##의', '##치', '##유', '##와', '##현', '##대', '##인의', '##고', '##독', '##을', '다', '##루', '##는', '##척', '##하는', ',', '[UNK]', '.', '선', '##인', '##장에', '##나', '찔', '##려', '##라']


In [ ]:
string = tokenizer.convert_tokens_to_string(tokens)
print(string)

영혼의치유와현대인의고독을 다루는척하는 , [UNK] . 선인장에나 찔려라


### 텍스트의 단어장은 만든 후 라벨 단어장은 만들기

In [ ]:
LABEL.build_vocab(train_data)

In [ ]:
print(LABEL.vocab.stoi)

defaultdict(None, {'0': 0, '1': 1})


### 이터레이터를 만들자. 배치 사이즈는 가능한 크게 하는게 트랜스포머의 성능을 올리는 데 도움이 되는 편

In [ ]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

# 모델 생성
* 토크나이저와 동일하게 모델도 사전 훈련된 모델을 불러오기

In [ ]:
from transformers import BertModel

bert = BertModel.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


HBox(children=(IntProgress(value=0, description='Downloading', max=714314041, style=ProgressStyle(description_…
이제 진짜 모델을 정의하자. 임베딩 레이어는 앞서 정의한 트랜스포머 모델로 대체한다. 이후 임베딩을 GRU 에 넣어서 결과를 출력하도록 한다. 이때 임베딩 차원(hidden_size)은 트랜스포머의 config에서 가져온다.

순전파 과정에서 트랜스포머를 no_grad로 감싸서 훈련되지 않도록 한다. 트랜스포머 모델은 원래 pooling된 결과값을 반환하게 되어 있는데, 여기서는 쓰지 않겠다. 이 외에는 일반적인 RNN 모델의 구조이다.

In [ ]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self, bert, hidden_dim, output_dim,
                n_layers, bidirectional, dropout):
        super().__init__()
        self.bert = bert
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.rnn = nn.GRU(embedding_dim, hidden_dim,
                         num_layers = n_layers,
                         bidirectional = bidirectional,
                         batch_first = True,
                         dropout = 0 if n_layers <2 else dropout)
        self.out = nn.Linear(hidden_dim * 2 if bidirectional
                            else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [batch_size, sent_len]
        with torch.no_grad():
            embedded = self.bert(text)[0]    
        #embedded = [batch_size, sent_len, emb_dim]

        _, hidden = self.rnn(embedded)
        #hideen = [n_layers * n_directions, batch_size, emb_dim]
        
        if self.rnn.bidirectional:
            # 마지막 레이어의 양방향 히든 벡터만 가져옴
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        #hidden = [batch_size, hid_dim]
        
        output = self.out(hidden)
        #output = [batch_size, out_dim]
        
        return output

### 하이퍼파라미터 지정 및 모델 생성

In [ ]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert, HIDDEN_DIM, OUTPUT_DIM,
                        N_LAYERS, BIDIRECTIONAL, DROPOUT)

### 모델의 파라미터 수 확인

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'모델의 파라미터 수는 {count_parameters(model):,}, 이 중 버트 모델의 파라미터 수는 {count_parameters(bert):,}개입니다.')

모델의 파라미터 수는 180,612,609, 이 중 버트 모델의 파라미터 수는 177,853,440개입니다.


### 버트 모델의 파라미터는 훈련시키지 않기 위해 얼리기

In [ ]:
for name, param in model.named_parameters():

    if name.startswith('bert'):
        param.requires_grad = False

###  실제 훈련시켜야 하는 모델의 파라미터 수를 확인

In [ ]:
print(f'모델의 파라미터 수는 {count_parameters(model):,}개입니다.')

모델의 파라미터 수는 2,759,169개입니다.


In [ ]:
for name, param in model.named_parameters():

    if param.requires_grad == True:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


# 모델 훈련

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds==y).float()
    acc = correct.sum() / len(correct)
    return acc

### 훈련함수 정의. model.train() 사용

In [ ]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1) # output_dim = 1
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### 훈련 시작

In [ ]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 523m 53s
	Train Loss: 0.523 | Train Acc: 73.32%
	 Val. Loss: 0.433 |  Val. Acc: 79.68%
Epoch: 02 | Epoch Time: 530m 48s
	Train Loss: 0.423 | Train Acc: 80.15%
	 Val. Loss: 0.390 |  Val. Acc: 81.85%


### 테스트셋 올려보기

In [ ]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

### 추가 훈련

In [ ]:
"""
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+6:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
"""

# 예측

In [ ]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, tokenizer, "이 영화 진짜 재밌었다!!")

In [ ]:

predict_sentiment(model, tokenizer, "이 영화 감독 밥은 먹고 다니냐? 이런 영화 만들고 잠이 와?")